In [1]:
import numpy as np
import pandas as pd
from sunpy.physics.differential_rotation import rot_hpc
from astropy import units as u
from shapely.geometry import Polygon, Point
from shapely import wkt
import os
import datetime
from sunpy.time import parse_time

In [2]:
#function to extract unique NOAA numbers from raw AR data and collect data points into individual files for each NOAA 
#numbered AR in a user named folder
def collect_ar_noaa(df, folder):
    #pick out unique entries in the ar_noaanum column
    unique_noaa = df.ar_noaanum.unique()
    #import a list of keywords for AR data entries as a list
    keywords_ar = list(np.genfromtxt('keywords_ar.csv', delimiter=',', dtype =str))
    #create a dictionary with unique NOAA numbers as keys 
    noaa_dic = {elem : pd.DataFrame for elem in unique_noaa}
    #create empty lists to fill while iterating over 
    start_times = []
    end_times= []
    file_names = []
    i = 0
    for key in noaa_dic.keys():
        if i != 0:
            noaa_dic[key] = df[:][df.ar_noaanum == key]
            start_time = noaa_dic[key]['event_starttime'].values[0]
            end_time = noaa_dic[key]['event_endtime'].values[-1]
            start_times.append(start_time)
            end_times.append(end_time)
            fName = str(key)[:-2] +'_'+ start_time[0:10]+'_'+end_time[0:10]
            location = folder + '/' + fName+ '.csv'
            noaa_dic[key].to_csv(location, columns = keywords_ar, index = False)
            file_names.append(fName)
        i+=1  

In [2]:
#import AR keywords as a numpy array
ar_keywords = np.genfromtxt('keywords_ar_1.csv', delimiter = ',', dtype = str)

In [8]:
def merge_ar_sources(fname, ar_keywords, output_directory):
    inputFile = 'AR_raw/'+fname
    ar_set =pd.read_csv(inputFile, delimiter = ',', header = 0)
    ar_set_NOAA = ar_set.ix[ar_set['frm_identifier']=='NOAA SWPC']
    listOindices = ar_set_NOAA.index.tolist()
    keywords_ar_add = ['ar_mcintoshcls', 'ar_mtwilsoncls', 'ar_numspots']
    ar_set.sort_values(by=['event_starttime','frm_identifier'], ascending =[True, False])
    if listOindices!= [] and listOindices!=range(0, ar_set.shape[0]):
        for i in range(0, listOindices[0]):
            for elem in keywords_ar_add:
                ar_set.loc[i, elem] = ar_set.loc[listOindices[0], elem] 
        for i in range(listOindices[-1], ar_set.shape[0]):
            for elem in keywords_ar_add:
                ar_set.loc[i, elem] = ar_set.loc[listOindices[-1], elem]
        for i, idx in enumerate(listOindices):
            if idx!=listOindices[-1]:
                for j in range(idx, listOindices[i+1]):
                    for elem in keywords_ar_add:
                        ar_set.loc[j, elem] = ar_set.loc[idx, elem]
        ar_set = ar_set.drop(listOindices)
        outputFile = output_directory + '/' + fname
        ar_set.to_csv(outputFile, index = False, columns = ar_keywords)
        
    return ar_set

In [9]:
AR_folder = 'AR4'
for root, dirs, files in os.walk('AR_raw'):
    for f in files:
        if f[0] != '.':
            _ = merge_ar_sources(f, ar_keywords, AR_folder)

In [3]:
def rotate_polygon(polygon, start_time, end_time):
    poly_coords = list(polygon.exterior.coords)
    poly_coords = (np.asarray(poly_coords[0:-1])) *u.arcsec
    poly_coords_x = poly_coords[:,0]
    poly_coords_y = poly_coords[:,1]
    rotated_poly_coords_x, rotated_poly_coords_y = rot_hpc(poly_coords_x, poly_coords_y, start_time, end_time)
    return poly_coords_x.value, poly_coords_y.value, rotated_poly_coords_x.value, rotated_poly_coords_y.value

def get_poly_coords(x, y, left_side = False):
    if left_side: diff = abs(y-y[x.argmin()])
    else: diff = abs(y-y[x.argmax()])
    d = {'x': x, 'y': y, 'diff':diff}
    sorter_df = pd.DataFrame(data=d)
    sorter_df = sorter_df.sort_values(by='x', ascending = left_side)
    x0 = sorter_df['x'].values[0]
    y0 = sorter_df['y'].values[0]
    if sorter_df['diff'].values[1] >= sorter_df['diff'].values[2]:
        x1 = sorter_df['x'].values[1]
        y1 = sorter_df['y'].values[1]
    else:
        x1 = sorter_df['x'].values[2]
        y1 = sorter_df['y'].values[2]
    return x0, y0, x1, y1

#function breaks down when AR rotating around the limb

def rotate_ar(polygon, start_time, end_time):
    og_x, og_y, rot_x, rot_y = rotate_polygon(polygon, start_time, end_time)
    x0, y0, x1, y1 = get_poly_coords(og_x, og_y, True)
    x2, y2, x3, y3 = get_poly_coords(rot_x, rot_y, False)
    #ensure that vertices of quadrilateral are in counterwise order in order to create proper Polygon object
    if y0 < y1:
        dummy_y = y0
        dummy_x = x0
        y0 = y1
        x0 = x1
        y1 = dummy_y
        x1 = dummy_x
    if y2 > y3:
        dummy_y = y2
        dummy_x = x2
        y2 = y3
        x2 = x3
        y3 = dummy_y
        x3 = dummy_x
    rotated_ar_polygon = Polygon([(x0,y0), (x1,y1), (x2,y2), (x3,y3)])
    return rotated_ar_polygon

In [ ]:
for root, dirs, files in os.walk(AR_folder):
    for f in files:
        if f[0] != '.':
            inputFile = AR_folder +'/'+f
            ars = pd.read_csv(inputFile, delimiter = ',', header = 0)
            ar_polygon = ars['hpc_bbox']
            start_time = ars['event_starttime']
            end_time = ars['event_endtime']
            full_hpc_polygon = []
            for i in range(ars.shape[0]):
                poly = rotate_ar(ar_polygon[i], start_time[i], end_time[i])
                full_hpc_polygon.append(poly)
            

In [8]:
def create_ar_list(AR_folder, outputF):
    file_names = []
    ar_nums = []
    start_years = []
    start_months = []
    start_days =[]
    start_times = []
    end_years = []
    end_months = []
    end_days = []
    end_times = []

    for root, dirs, files in os.walk(AR_folder):
        for f in files:
            if f[0] != '.':
                file_names.append(f)
                ar_num = f[0:5]
                ar_nums.append(ar_num)
                start_year = f[6:10]
                start_years.append(start_year)
                start_month = f[11:13]
                start_months.append(start_month)
                start_day = f[14:16]
                start_days.append(start_day)
                start_time = f[6:16]
                start_times.append(start_time)
                end_year = f[17:21]
                end_years.append(end_year)
                end_month = f[22:24]
                end_months.append(end_month)
                end_day = f[25:27]
                end_days.append(end_day)
                end_time = f[17:27]
                end_times.append(end_time)

    dic4df = {'file_name': file_names, 'ar_num': ar_nums,'start_year': start_years,'start_month': start_months,
              'start_day': start_days,'start_time': start_times, 'end_year': end_years,'end_month':end_months,
              'end_day': end_days,'end_time': end_times}
    ar_list = pd.DataFrame(data=dic4df)
    ar_list.to_csv(outputF, index = False)
        

In [6]:
def associate_ar(inputFile_fl, inputFile_ar, ar_folder, output2file=False, out_file = None):
    #import a record of flare events as a DataFrame
    flare_set = pd.read_csv(inputFile_fl, delimiter = ',', header = 0)
    #import a list of ar events as a DataFrame
    ar_list = pd.read_csv(inputFile_ar, delimiter = ',', header = 0)
    
    
    #how many flare events working with
    length = flare_set.shape[0]
    #list of zeroes with length of number of flare events
    zeroes = [0 for i in range(length)]
    #list of nulls with length of number of flare events
    nones = [None for i in range(length)]
    #create columns (filled with zeroes) for tracking associated events
    flare_set.loc[:, 'associated_ar'] = nones
    #get a list of ar keywords of relevance to flare events
    ar_keywords = list(np.genfromtxt('keywords_ar_append_fl.csv', delimiter=',', dtype=str))
    #create columns filled with Nones for each ar keyword
    for elem in ar_keywords:
         flare_set.loc[:, elem] = nones
    #convert start and end times to datetime objects
    flare_set['event_starttime'] = map(parse_time, flare_set['event_starttime'])
    flare_set['event_endtime'] = map(parse_time, flare_set['event_endtime'])
#     ar_list['start_time'] = map(parse_time, ar_list['start_time'])
#     ar_list['end_time'] = map(parse_time, ar_list['end_time'])
    ar_list['start_time'] = map(lambda x, y, z: datetime.datetime(x, y, z), ar_list['start_year'], 
                                ar_list['start_month'], ar_list['start_day'])
    ar_list['end_time'] = map(lambda x, y, z: datetime.datetime(x, y, z), ar_list['end_year'], 
                                  ar_list['end_month'], ar_list['end_day'])
    #set positional row index 
    i = -1

    for annoying_wrong_obj_type in flare_set['event_starttime']:
        i += 1
        #print which flare event function is currently processing, so the user has an idea of how much longer
        #program will need to run
        start = flare_set['event_starttime'].values[i]
        end = flare_set['event_endtime'].values[i]
        if (i+1)%25 == 0:
            print '%d / %d events' %((i+1), length)
        #begin eliminating ar events based on temporal parameters
        ar_search = ar_list.ix[ar_list['start_time']<=start]
        ar_search = ar_search.ix[ar_search['end_time']>=end]
        ar_files = list(ar_search['file_name'])
        
        #as long as the temporal search does not eliminate all possible related AR events, proceed
        if len(ar_files)!= 0:
            for f in ar_files:
                fname = ar_folder+'/'+f
                specific_ar = pd.read_csv(fname, delimiter = ',', header = 0)
                specific_ar = specific_ar.rename(columns={'hpc_bbox': 'ar_hpc_bbox'})
                found_time_match = False
                specific_ar['event_starttime'] = map(parse_time, specific_ar['event_starttime'])
                specific_ar['event_endtime'] = map(parse_time, specific_ar['event_endtime'])
                j = 0
                while found_time_match==False and j<len(specific_ar):
#                     print type(specific_ar['event_starttime'].values[j])
#                     print type(start)
#                     print type(specific_ar['event_endtime'].values[j])
                    if (specific_ar['event_starttime'].values[j] <= start and 
                        specific_ar['event_endtime'].values[j] >= start):
                        found_time_match == True
#                         print start
#                         print end
#                         print annoying_wrong_obj_type
                        fl_point = Point((flare_set['hpc_x'].values[i], flare_set['hpc_y'].values[i]))
                        ar_poly_og = wkt.loads(specific_ar['ar_hpc_bbox'].values[j])
                        end = specific_ar.loc[j, 'event_endtime']
                        ar_poly = rotate_ar(ar_poly_og, annoying_wrong_obj_type, end)
                        if fl_point.intersects(ar_poly):
                            flare_set['associated_ar'].values[i] = str(specific_ar.loc[j, 'SOL_standard'])
                            for elem in ar_keywords:
                                flare_set.loc[i, elem] = specific_ar[elem].values[j]
                    j+=1
            
    #create boolean var to easily determine whether flare associated with an AR
    k = 0
    is_ar = [0 for i in range(flare_set.shape[0])]
    for elem in flare_set['associated_ar']:
        if elem!=None:
            is_ar[k] = 1
        k+=1
    flare_set.loc[:, 'is_ar'] = is_ar
    
    #write dataframe to a csv file depending on initial parameters
    if output2file:
        if out_file == None:
            #create a generic name for file based on search parameters if no file name specified
             out_file = inputFile_fl[0:-4]+'_with_ar.csv'
        #import which keywords to keep for outported data
        flare_keywords = list(np.genfromtxt('keywords_flare_with_goes.csv', delimiter=',', dtype=str))
        #add to these keywords descriptors of associated AR
        flare_keywords.extend(['is_ar','associated_ar'])
        flare_keywords.extend(ar_keywords)
        #write to csv
        flare_set.to_csv(path_or_buf=out_file, columns = flare_keywords, index = False)
        
    return flare_set

In [7]:
associate_ar('flare_dataset_cleaned_30min_100arcsec_with_GOES.csv', 'ar4_list.csv', 'AR4', output2file=True, 
             out_file = None)

25 / 40293 events
50 / 40293 events
75 / 40293 events
100 / 40293 events
125 / 40293 events
150 / 40293 events
175 / 40293 events
200 / 40293 events
225 / 40293 events
250 / 40293 events
275 / 40293 events
300 / 40293 events
325 / 40293 events
350 / 40293 events
375 / 40293 events
400 / 40293 events
425 / 40293 events
450 / 40293 events
475 / 40293 events
500 / 40293 events
525 / 40293 events
550 / 40293 events
575 / 40293 events
600 / 40293 events
625 / 40293 events
650 / 40293 events
675 / 40293 events
700 / 40293 events
725 / 40293 events
750 / 40293 events
775 / 40293 events
800 / 40293 events
825 / 40293 events
850 / 40293 events
875 / 40293 events
900 / 40293 events
925 / 40293 events
950 / 40293 events
975 / 40293 events
1000 / 40293 events
1025 / 40293 events
1050 / 40293 events
1075 / 40293 events
1100 / 40293 events
1125 / 40293 events
1150 / 40293 events
1175 / 40293 events
1200 / 40293 events
1225 / 40293 events
1250 / 40293 events
1275 / 40293 events
1300 / 40293 events
13

,SOL_standard,event_starttime,event_endtime,event_peaktime,hpc_bbox,hpc_coord,hpc_radius,hpc_x,hpc_y,is_associated_fl,...,meanenergydensityunit,totalenergydensityunit,totalphotoenergy,totalphotoenergyunit,unsignedflux,magfluxunit,highsheararea,highshearareaunit,ar_hpc_bbox,is_ar
0,SOL2010-06-11T20:27:24L105C069,2010-06-11 20:27:24,2010-06-11 21:02:48,2010-06-11T20:37:36,"POLYGON((614.4 307.2,691.2 307.2,691.2 384,614...",POINT(652.8 345.6),738.638748,652.8,345.6,1,...,ergs per cubic centimeter,ergs/cm,1.88604e+30,ergs,4737.1,emx,1.22199e+10,km2,"POLYGON((533.2764 333.4344,651.834 334.4202,62...",1
1,SOL2010-06-11T20:27:26L105C069,2010-06-11 20:27:26,2010-06-12 14:31:26,2010-06-12T01:00:14,"POLYGON((614.4 307.2,768 307.2,768 460.8,614.4...",POINT(652.8 345.6),738.638748,652.8,345.6,1,...,ergs per cubic centimeter,ergs/cm,1.88604e+30,ergs,4737.1,emx,1.22199e+10,km2,"POLYGON((533.2764 333.4344,651.834 334.4202,62...",1
2,SOL2010-06-12T05:20:39L100C069,2010-06-12 05:20:39,2010-06-12 05:27:03,2010-06-12T05:21:39,"POLYGON((614.4 307.2,691.2 307.2,691.2 384,614...",POINT(652.8 345.6),738.638748,652.8,345.6,1,...,ergs per cubic centimeter,ergs/cm,2.17185e+30,ergs,7332.33,emx,1.26973e+10,km2,"POLYGON((578.2764 324.369,713.37 325.8066,679....",1
3,SOL2010-06-12T09:04:03L098C069,2010-06-12 09:04:03,2010-06-12 09:29:27,2010-06-12T09:14:27,"POLYGON((614.4 307.2,768 307.2,768 460.8,614.4...",POINT(652.8 345.6),738.638748,652.8,345.6,1,...,ergs per cubic centimeter,ergs/cm,2.43671e+30,ergs,8438.56,emx,8.26814e+09,km2,"POLYGON((600.348 324.2352,736.116 325.854,699....",1
4,SOL2010-06-13T05:35:35L128C115,2010-06-13 05:35:35,2010-06-13 05:50:47,2010-06-13T05:40:59,"POLYGON((844.8 -460.8,921.6 -460.8,921.6 -307....",POINT(883.2 -422.4),979.011747,883.2,-422.4,1,...,None,None,None,None,None,None,None,None,None,0
5,SOL2010-06-13T05:54:48L105C069,2010-06-13 05:54:48,2010-06-13 05:58:24,2010-06-13T05:56:12,"POLYGON((768 307.2,844.8 307.2,844.8 384,768 3...",POINT(806.4 345.6),877.337062,806.4,345.6,1,...,ergs per cubic centimeter,ergs/cm,4.41394e+30,ergs,11151.3,emx,2.49581e+10,km2,"POLYGON((707.388 320.826,829.266 323.4732,787....",1
6,SOL2010-06-13T07:05:36L110C116,2010-06-13 07:05:36,2010-06-13 07:16:00,2010-06-13T07:08:48,"POLYGON((768 -460.8,844.8 -460.8,844.8 -384,76...",POINT(806.4 -422.4),910.331105,806.4,-422.4,1,...,None,None,None,None,None,None,None,None,None,0
7,SOL2010-06-13T07:33:00L097C064,2010-06-13 07:33:00,2010-06-13 07:38:24,2010-06-13T07:34:48,"POLYGON((691.2 307.2,768 307.2,768 460.8,691.2...",POINT(729.6 422.4),843.052739,729.6,422.4,1,...,ergs per cubic centimeter,ergs/cm,4.41394e+30,ergs,11151.3,emx,2.49581e+10,km2,"POLYGON((707.388 320.826,829.266 323.4732,787....",1
8,SOL2010-06-13T08:09:39L104C069,2010-06-13 08:09:39,2010-06-13 08:17:51,2010-06-13T08:13:03,"POLYGON((768 307.2,844.8 307.2,844.8 384,768 3...",POINT(806.4 345.6),877.337062,806.4,345.6,1,...,ergs per cubic centimeter,ergs/cm,6.14796e+30,ergs,12434.9,emx,3.48456e+10,km2,"POLYGON((725.4 319.0356,843.708 321.9564,800.9...",1
9,SOL2010-06-13T09:39:26L103C069,2010-06-13 09:39:26,2010-06-13 09:56:26,2010-06-13T09:45:38,"POLYGON((768 307.2,844.8 307.2,844.8 384,768 3...",POINT(806.4 345.6),877.337062,806.4,345.6,1,...,ergs per cubic centimeter,ergs/cm,6.14796e+30,ergs,12434.9,emx,3.48456e+10,km2,"POLYGON((725.4 319.0356,843.708 321.9564,800.9...",1


In [ ]:
time_zero = start_time
    delta_t = lightcurve_search['DATE_OBS'] - time_zero

In [1]:
print(__doc__)

# Author: Gael Varoquaux <gael dot varoquaux at normalesup dot org>
# License: BSD 3 clause

# Standard scientific Python imports
import matplotlib.pyplot as plt

# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics

# The digits dataset
digits = datasets.load_digits()

# The data that we are interested in is made of 8x8 images of digits, let's
# have a look at the first 3 images, stored in the `images` attribute of the
# dataset.  If we were working from image files, we could load them using
# pylab.imread.  Note that each image must have the same size. For these
# images, we know which digit they represent: it is given in the 'target' of
# the dataset.
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[:4]):
    plt.subplot(2, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %i' % label)

# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
classifier = svm.SVC(gamma=0.001)

# We learn the digits on the first half of the digits
classifier.fit(data[:n_samples / 2], digits.target[:n_samples / 2])

# Now predict the value of the digit on the second half:
expected = digits.target[n_samples / 2:]
predicted = classifier.predict(data[n_samples / 2:])

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

images_and_predictions = list(zip(digits.images[n_samples / 2:], predicted))
for index, (image, prediction) in enumerate(images_and_predictions[:4]):
    plt.subplot(2, 4, index + 5)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Prediction: %i' % prediction)

plt.show()


Automatically created module for IPython interactive environment
Classification report for classifier SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False):
             precision    recall  f1-score   support

          0       1.00      0.99      0.99        88
          1       0.99      0.97      0.98        91
          2       0.99      0.99      0.99        86
          3       0.98      0.87      0.92        91
          4       0.99      0.96      0.97        92
          5       0.95      0.97      0.96        91
          6       0.99      0.99      0.99        91
          7       0.96      0.99      0.97        89
          8       0.94      1.00      0.97        88
          9       0.93      0.98      0.95        92

avg / total       0.97      0.97      0.97       899


Confusion matrix:
[[87  0  0  0  1

7
